In [1]:
import requests
import pandas as pd
import plotly.express as px
import time

# 🔑 Keys
SPOTIFY_KEY = "cc5dbd39eamshe144a9f01c013e7p114b53jsn4200d3c48103"
SPOTIFY_HOST = "spotify23.p.rapidapi.com"
SPOTIFY_HEADERS = {"x-rapidapi-key": SPOTIFY_KEY, "x-rapidapi-host": SPOTIFY_HOST}

RECOBEATS_KEY = "YOUR_RECOBEATS_API_KEY"   # <-- replace
RECOBEATS_URL = "https://track-analysis.p.rapidapi.com/pktx/rapid"
RECOBEATS_HEADERS = {
    "x-rapidapi-key": RECOBEATS_KEY,
    "x-rapidapi-host": "track-analysis.p.rapidapi.com"
}



In [2]:
# 🎤 Search Artist
def search_artist(name):
    url = f"https://{SPOTIFY_HOST}/search/"
    params = {"q": name, "type": "artist", "limit": "1"}
    res = requests.get(url, headers=SPOTIFY_HEADERS, params=params).json()
    artist = res["artists"]["items"][0]["data"]
    return artist["uri"].split(":")[-1], artist["profile"]["name"]

artist_id, artist_name = search_artist("Arijit Singh")
print("🎤 Artist:", artist_name, "| ID:", artist_id)

# 📀 Get Albums
def get_albums(artist_id):
    url = f"https://{SPOTIFY_HOST}/artist_albums/"
    params = {"id": artist_id, "limit": "20"}
    res = requests.get(url, headers=SPOTIFY_HEADERS, params=params).json()
    albums = []
    for item in res["data"]["artist"]["discography"]["albums"]["items"]:
        album = item["releases"]["items"][0]
        albums.append({
            "name": album["name"],
            "release_date": int(album["date"]["year"]),
            "tracks": album["tracks"]["totalCount"]
        })
    return pd.DataFrame(albums)

albums_df = get_albums(artist_id)


🎤 Artist: Arijit Singh | ID: 4YRxDV8wJFPHPTeXepOstw


In [10]:
# 🎶 Get Top Tracks
def get_top_tracks(artist_id):
    url = f"https://{SPOTIFY_HOST}/artist_overview/"
    params = {"id": artist_id}
    res = requests.get(url, headers=SPOTIFY_HEADERS, params=params).json()

    tracks = []
    try:
        for item in res["data"]["artist"]["discography"]["topTracks"]["items"]:
            track = item["track"]
            tracks.append({
                "name": track.get("name"),
                "playcount": int(track.get("playcount", 0)),
                "id": track.get("id")
            })
    except KeyError:
        print("⚠️ Could not find topTracks in response.")
    return pd.DataFrame(tracks)

# Run again
tracks_df = get_top_tracks(artist_id)
print(tracks_df.head())

                                           name  playcount  \
0                        Dhun (From "Saiyaara")   52568058   
1                                  Apna Bana Le  494139826   
2                                  Aavan Jaavan   21733828   
3  Tujhe Kitna Chahne Lage (From "Kabir Singh")  586543820   
4                                     Tum Hi Ho  421416944   

                       id  
0  1GFQthxDi4Tqe3wJ5j0G5S  
1  1hA697u7e1jX2XM8sWA6Uy  
2  3x822BpQYSFMIB7P3uiJN0  
3  2Fv2injs4qAm8mJBGaxVKU  
4  56zZ48jdyY2oDXHVnwg5Di  


In [11]:
# 🎼 Fetch Audio Features from RecoBeats
def get_audio_features(track_name, artist_name):
    params = {"song": track_name, "artist": artist_name}
    try:
        res = requests.get(RECOBEATS_URL, headers=RECOBEATS_HEADERS, params=params, timeout=10).json()
        return {
            "danceability": res.get("danceability", None),
            "energy": res.get("energy", None),
            "tempo": res.get("tempo", None)
        }
    except Exception as e:
        print("⚠️ Error fetching audio features:", e)
        return {"danceability": None, "energy": None, "tempo": None}

# Merge features into Top Tracks
features_list = []
for _, row in tracks_df.iterrows():
    f = get_audio_features(row["name"], artist_name)
    row_data = row.to_dict()
    row_data.update(f)
    features_list.append(row_data)
    time.sleep(1)  # avoid hitting API too fast

tracks_features_df = pd.DataFrame(features_list)

In [12]:
# 📊 Scatter Plot: Albums
if not albums_df.empty:
    fig1 = px.scatter(albums_df, x="release_date", y="tracks",
                      size="tracks", color="release_date",
                      hover_name="name",
                      title=f"📀 Albums of {artist_name}: Release Year vs Track Count")
    fig1.show()

In [14]:
# 📊 Bar Plot: Top Tracks
if not tracks_df.empty:
    fig2 = px.bar(
        tracks_df,
        x="name",
        y="playcount",   # ✅ FIXED HERE
        title=f"🎶 Top Tracks of {artist_name}",
        labels={"name": "Track", "playcount": "Play Count"},
        color="playcount"
    )
    fig2.update_layout(xaxis_tickangle=-45)
    fig2.show()